In [1]:
from deepface import DeepFace


2024-06-28 09:38:46.412967: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-28 09:38:47.399867: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
models = ["Facenet512"]
models[0]

'Facenet512'

In [14]:
import pandas as pd
import cv2
from deepface import DeepFace
import gradio as gr

def find_similar_faces(input_image_path, db_path):
    # Find similar faces
    dfs = DeepFace.find(
        img_path=input_image_path,
        db_path=db_path,
        enforce_detection=False,
        model_name=models[0]
    )
   
    # Convert the result to a pandas DataFrame if it's not already
    if isinstance(dfs, list):
        df = dfs[0]
    else:
        df = dfs
   
    # Sort by similarity (assuming lower distance means more similar)
    df_sorted = df.sort_values(by=['distance'])
   
    # Get top 3 results
    top_3 = df_sorted.head(3)
   
    return top_3

def visualize_results(top_3):
    # Create a list to store similar faces
    images = []
   
    # Add top 3 similar faces to the list
    for _, row in top_3.iterrows():
        img = cv2.imread(row['identity'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images.append(img)
   
    return images

def process_image(input_image):
    # Save the input image temporarily
    input_path = "/teamspace/studios/this_studio/meriphoto.jpg"
    input_image.save(input_path)
    
    # Find similar faces
    db_path = "/teamspace/studios/this_studio/data/facedata/train"
    top_3 = find_similar_faces(input_path, db_path)
    
    # Visualize results (only similar faces)
    result_images = visualize_results(top_3)
    
    return result_images

# Create Gradio interface
iface = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="pil"),
    outputs=[gr.Image(type="numpy") for _ in range(3)],  # Only 3 similar faces
    title="Find Similar Faces",
    description="Upload an image to find the top 3 similar faces."
)

# Launch the interface
iface.launch()

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


In [ ]:
import os
import shutil
import gradio as gr
from deepface import DeepFace
import cv2
import pandas as pd

# Assume models is defined somewhere
models = ["Facenet512","VGG-Face", "Facenet", "OpenFace", "DeepFace", "DeepID", "Dlib", "ArcFace"]

def save_training_images(name, images):
    db_path = "/teamspace/studios/this_studio/data/facedata/train"
    person_folder = os.path.join(db_path, name)
    
    # Create folder if it doesn't exist
    os.makedirs(person_folder, exist_ok=True)
    
    # Save images
    for i, img in enumerate(images):
        img_path = os.path.join(person_folder, f"{name}_{i}.jpg")
        img.save(img_path)
    
    return f"Saved {len(images)} images for {name}"

def find_similar_faces(input_image_path, db_path):
    dfs = DeepFace.find(
        img_path=input_image_path,
        db_path=db_path,
        enforce_detection=False,
        model_name=models[0]
    )
    
    if isinstance(dfs, list):
        df = dfs[0]
    else:
        df = dfs
    
    df_sorted = df.sort_values(by=['distance'])
    top_3 = df_sorted.head(3)
    
    return top_3

def visualize_results(top_3):
    images = []
    
    for _, row in top_3.iterrows():
        img = cv2.imread(row['identity'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images.append(img)
    
    return images

def process_image(input_image):
    input_path = "/teamspace/studios/this_studio/meriphoto.jpg"
    input_image.save(input_path)
    
    db_path = "/teamspace/studios/this_studio/data/facedata/train"
    top_3 = find_similar_faces(input_path, db_path)
    
    result_images = visualize_results(top_3)
    
    return result_images

# Create Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("# Find Similar Faces")
    
    with gr.Tab("Find Similar Faces"):
        input_image = gr.Image(type="pil")
        find_button = gr.Button("Find Similar Faces")
        output_images = [gr.Image(type="numpy") for _ in range(3)]
        
        find_button.click(process_image, inputs=input_image, outputs=output_images)
    
    with gr.Tab("Add Training Data"):
        name_input = gr.Textbox(label="Name")
        image_inputs = [gr.Image(type="pil") for _ in range(4)]
        train_button = gr.Button("Add Training Data")
        train_output = gr.Textbox(label="Training Result")
        
        train_button.click(save_training_images, inputs=[name_input] + image_inputs, outputs=train_output)

# Launch the interface
iface.launch()

# Start from here

In [3]:
import os
import shutil
import gradio as gr
from deepface import DeepFace
import cv2
import pandas as pd

In [4]:
import traceback

def save_training_images(name, images):
    db_path = "/teamspace/studios/this_studio/data/facedata/train"
    person_folder = os.path.join(db_path, name)
    
    try:
        # Create folder if it doesn't exist
        os.makedirs(person_folder, exist_ok=True)
    except Exception as e:
        error_message = f"Error creating directory {person_folder}: {str(e)}\n{traceback.format_exc()}"
        raise gr.Error(f"{error_message} 💥", duration=5)
    
    try:
        # Save images
        for i, img in enumerate(images):
            img_path = os.path.join(person_folder, f"{name}_{i}.jpg")
            img.save(img_path)
    except Exception as e:
        error_message = f"Error saving images: {str(e)}\n{traceback.format_exc()}"
        raise gr.Error(f"{error_message} 💥", duration=5)
    
    return f"Saved {len(images)} images for {name}"

In [5]:

models = ["Facenet512", "VGG-Face", "Facenet", "OpenFace", "DeepFace", "DeepID", "Dlib", "ArcFace"]


def find_similar_faces(input_image_path, db_path):
    try:
        dfs = DeepFace.find(
            img_path=input_image_path,
            db_path=db_path,
            enforce_detection=False,
            model_name=models[0]
        )
        
        if isinstance(dfs, list):
            df = dfs[0]
        else:
            df = dfs
        
        df_sorted = df.sort_values(by=['distance'])
        top_3 = df_sorted.head(3)
        
        return top_3
    except Exception as e:
        raise gr.Error(f"Error finding similar faces: {str(e)} 💥", duration=5)

In [6]:
def visualize_results(top_3):
    images = []
    
    for _, row in top_3.iterrows():
        img = cv2.imread(row['identity'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images.append(img)
    
    return images

In [7]:

def process_image(input_image):
    input_path = "/teamspace/studios/this_studio/meriphoto.jpg"
    try:
        input_image.save(input_path)
    except Exception as e:
        raise gr.Error(f"Error saving input image: {str(e)} 💥", duration=5)
    
    db_path = "/teamspace/studios/this_studio/data/facedata/train"
    top_3 = find_similar_faces(input_path, db_path)
    
    if top_3.empty:
        return [None, None, None]
    
    result_images = visualize_results(top_3)
    
    return result_images






In [8]:
# Create Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("# Find Similar Faces")
    
    with gr.Tab("Find Similar Faces"):
        input_image = gr.Image(type="pil")
        find_button = gr.Button("Find Similar Faces")
        output_images = [gr.Image(type="numpy") for _ in range(3)]
        
        find_button.click(process_image, inputs=input_image, outputs=output_images)
    
    with gr.Tab("Add Training Data"):
        name_input = gr.Textbox(label="Name")
        image_inputs = [gr.Image(type="pil") for _ in range(4)]
        train_button = gr.Button("Add Training Data")
        train_output = gr.Textbox(label="Training Result")
        
        train_button.click(save_training_images, inputs=[name_input] + image_inputs, outputs=train_output)

In [9]:
# Launch the interface
iface.launch(share="True")

Running on local URL:  http://127.0.0.1:7860


Running on public URL: https://ec0548c70b1fd6e8b4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


24-06-28 09:40:06 - Searching /teamspace/studios/this_studio/meriphoto.jpg in 351 length datastore
24-06-28 09:40:16 - find function duration 9.56679391860962 seconds
24-06-28 09:41:55 - Searching /teamspace/studios/this_studio/meriphoto.jpg in 351 length datastore
24-06-28 09:42:02 - find function duration 6.439059495925903 seconds


Traceback (most recent call last):
  File "/tmp/ipykernel_204748/4072924861.py", line 4, in process_image
    input_image.save(input_path)
AttributeError: 'NoneType' object has no attribute 'save'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/gradio/queueing.py", line 541, in process_events
    response = await route_utils.call_process_api(
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/gradio/blocks.py", line 1928, in process_api
    result = await self.call_function(
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/gradio/blocks.py", line 1514, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/z

24-06-28 09:42:50 - Searching /teamspace/studios/this_studio/meriphoto.jpg in 351 length datastore
24-06-28 09:42:55 - find function duration 5.152935028076172 seconds


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/uvicorn/protocols/http/httptools_impl.py", line 399, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/uvicorn/middleware/proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/home/zeus/miniconda3/envs/cloudspace/l